## Get bugs information from `.csv` files

The following part extracts information from the `.csv` files dumped during running test suites on DBMSs.

In [31]:
import pandas as pd

bugs_df = pd.read_csv("output/mysql_bugs.zip",compression='zip')
logs_df = pd.read_csv("output/mysql_logs.zip", compression='zip')

print(bugs_df.info())
print(logs_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9951 entries, 0 to 9950
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       9951 non-null   int64  
 1   DBMS_NAME        9951 non-null   object 
 2   TESTFILE_INDEX   9951 non-null   int64  
 3   TESTFILE_PATH    9951 non-null   object 
 4   TESTCASE_INDEX   9951 non-null   int64  
 5   ERROR_SQL        9951 non-null   object 
 6   CASE_TYPE        9951 non-null   object 
 7   EXPECTED_RESULT  9670 non-null   object 
 8   ACTUAL_RESULT    9670 non-null   object 
 9   LOGS_INDEX       9951 non-null   int64  
 10  DATE             9951 non-null   object 
 11  EXEC_TIME        9951 non-null   float64
dtypes: float64(1), int64(4), object(7)
memory usage: 933.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  ----

In [32]:
from random import randint

id = randint(0, len(bugs_df))
bug_row = bugs_df.loc[id]
log_row = logs_df.loc[int(bug_row.LOGS_INDEX)]
print(logs_df.LOGS)
dbms_name = 'mysql'  # TODO need add a dbms_name field in the dataframe
test_suite = 'sqlite_tests'  # TODO same as above


0                                                   NaN
1     CREATE TABLE tab0(col0 INTEGER, col1 INTEGER, ...
2     CREATE TABLE tab0(col0 INTEGER, col1 INTEGER, ...
3     CREATE TABLE tab0(col0 INTEGER, col1 INTEGER, ...
4     CREATE TABLE tab0(col0 INTEGER, col1 INTEGER, ...
5     CREATE TABLE tab0(col0 INTEGER, col1 INTEGER, ...
6     CREATE TABLE tab0(col0 INTEGER, col1 INTEGER, ...
7     CREATE TABLE tab0(col0 INTEGER, col1 INTEGER, ...
8     CREATE TABLE tab0(col0 INTEGER, col1 INTEGER, ...
9     CREATE TABLE tab0(col0 INTEGER, col1 INTEGER, ...
10    CREATE TABLE tab0(col0 INTEGER, col1 INTEGER, ...
11    CREATE TABLE tab0(col0 INTEGER, col1 INTEGER, ...
12    CREATE TABLE tab0(col0 INTEGER, col1 INTEGER, ...
Name: LOGS, dtype: object


## Reproduce an error testcase

The following code picks a bug record in the `.csv` file and run on the selected db. Maybe running using the testrunner.

In [33]:
from src import testrunner

if dbms_name == 'sqlite':
    r = testrunner.SQLiteRunner()
elif dbms_name == 'duckdb':
    r = testrunner.DuckDBRunner()
elif dbms_name == 'cockroachdb':
    r = testrunner.CockroachDBRunner()
elif dbms_name == 'mysql':
    r = testrunner.MySQLRunner()
else:
    exit("Not implement yet")
    
r.connect("tempdb")

print(bug_row)
print(log_row)
print(log_row.LOGS)
r.executemany_stmt(log_row.LOGS)


print("The Expected Result is:")
print(bug_row.EXPECTED_RESULT)
print("The Actual Result is:")
result = r.execute_query(bug_row.ERROR_SQL)
for row in result:
    for item in row:
        print(item)






Unnamed: 0                                                  886
DBMS_NAME                                                 mysql
TESTFILE_INDEX                                                0
TESTFILE_PATH      sqlite_tests/random/select/slt_good_104.test
TESTCASE_INDEX                                             1073
ERROR_SQL           SELECT DISTINCT 42 * col0 FROM tab1 AS cor0
CASE_TYPE                                                 Query
EXPECTED_RESULT                                 126\n2688\n3360
ACTUAL_RESULT                                   126\n2688\n3360
LOGS_INDEX                                                   12
DATE                                             22-10-29-13:41
EXEC_TIME                                                 524.0
Name: 886, dtype: object
Unnamed: 0                                                   12
BUGS_INDEX                                                  NaN
LOGS          CREATE TABLE tab0(col0 INTEGER, col1 INTEGER, ...
Name: 12, dtype